# Day14_1: 차원 축소 (Dimensionality Reduction)

## 학습 목표

**Part 1: PCA (주성분 분석)**
1. 차원의 저주 개념 이해하기
2. PCA의 원리와 수학적 배경 이해하기
3. 설명된 분산(explained variance) 해석하기
4. 최적 주성분 개수 선택하기
5. Plotly로 2D/3D 시각화하기

**Part 2: t-SNE와 특성 선택**
1. t-SNE로 고차원 데이터 시각화하기
2. perplexity 파라미터 이해하기
3. SelectKBest로 통계적 특성 선택하기
4. RFE로 재귀적 특성 제거하기
5. Feature Importance 비교하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 차원 축소 | 고차원 데이터 압축 | 이미지 압축, 노이즈 제거 |
| PCA | 특성 추출, 전처리 | 얼굴 인식, 센서 데이터 분석 |
| t-SNE | 데이터 시각화 | 군집 탐색, 이상치 발견 |
| 특성 선택 | 모델 성능 향상 | 불필요한 변수 제거, 과적합 방지 |

**분석가 관점**: 100개의 특성을 가진 데이터를 2개로 줄여 시각화하고, 핵심 특성만 선택해 모델 성능을 높입니다!

---

In [ ]:
# 필요한 라이브러리 설치 및 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# scikit-learn
from sklearn.datasets import load_iris, load_digits, load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

---

# Part 1: PCA (주성분 분석)

---

## 1.1 차원의 저주 (Curse of Dimensionality)

### 차원의 저주란?

차원(특성 수)이 증가하면 발생하는 문제들:

1. **데이터 희소성**: 고차원 공간에서 데이터 포인트들이 멀어짐
2. **거리 의미 상실**: 모든 점들의 거리가 비슷해짐
3. **과적합 위험**: 특성 수 > 샘플 수이면 과적합
4. **계산 비용 증가**: 차원에 따라 기하급수적 증가

In [ ]:
# 차원의 저주 시각화: 차원별 최소-최대 거리 비율
np.random.seed(42)

dimensions = [2, 5, 10, 20, 50, 100, 200, 500]
n_samples = 100
distance_ratios = []

for dim in dimensions:
    # 고차원 데이터 생성
    data = np.random.randn(n_samples, dim)
    
    # 모든 쌍의 거리 계산
    from sklearn.metrics.pairwise import euclidean_distances
    dists = euclidean_distances(data).flatten()
    dists = dists[dists > 0]  # 자기 자신 제외
    
    # 최대-최소 거리 비율
    ratio = (dists.max() - dists.min()) / dists.min()
    distance_ratios.append(ratio)

# 시각화
fig = px.line(
    x=dimensions, 
    y=distance_ratios,
    markers=True,
    title='차원의 저주: 차원 증가에 따른 거리 비율 감소',
    labels={'x': '차원 수', 'y': '(최대거리-최소거리)/최소거리'}
)
fig.add_annotation(
    x=200, y=distance_ratios[dimensions.index(200)],
    text="거리 비율 감소 = 거리 구분 어려움",
    showarrow=True, arrowhead=2
)
fig.show()

### 실무 예시: 언제 차원 축소가 필요한가?

| 상황 | 특성 수 | 샘플 수 | 문제 | 해결책 |
|------|---------|---------|------|--------|
| 텍스트 분석 | 10,000+ (단어) | 1,000 | 희소 행렬 | TF-IDF + SVD |
| 이미지 분류 | 784 (28x28 픽셀) | 10,000 | 계산 비용 | PCA 압축 |
| 센서 데이터 | 500 (센서) | 5,000 | 다중공선성 | PCA/특성 선택 |
| 유전자 분석 | 20,000+ (유전자) | 200 | 과적합 | 특성 선택 |

---

## 1.2 PCA 개념과 원리

### PCA(Principal Component Analysis)란?

**주성분 분석**: 데이터의 분산을 최대화하는 방향(축)을 찾아 차원을 축소

**핵심 아이디어**:
1. 데이터를 가장 잘 설명하는 축(PC1) 찾기
2. PC1과 직교하며 다음으로 분산이 큰 축(PC2) 찾기
3. 이 과정을 반복하여 주성분들을 생성

**수학적 과정**:
1. 데이터 표준화 (평균 0, 분산 1)
2. 공분산 행렬 계산
3. 고유값/고유벡터 분해
4. 상위 k개 고유벡터 선택
5. 데이터를 새 축으로 투영

In [ ]:
# Iris 데이터로 PCA 시연
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print("Iris 데이터 형태:")
print(f"  특성 수: {X.shape[1]}개")
print(f"  샘플 수: {X.shape[0]}개")
print(f"  특성명: {feature_names}")
print(f"  클래스: {target_names}")

In [ ]:
# Step 1: 데이터 표준화 (필수!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("표준화 전 평균:", X.mean(axis=0).round(2))
print("표준화 후 평균:", X_scaled.mean(axis=0).round(4))
print("표준화 후 표준편차:", X_scaled.std(axis=0).round(4))

In [ ]:
# Step 2: PCA 적용
pca = PCA(n_components=2)  # 2개의 주성분으로 축소
X_pca = pca.fit_transform(X_scaled)

print("PCA 결과 형태:", X_pca.shape)
print("\n설명된 분산 비율:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.4f} ({var*100:.2f}%)")
print(f"\n누적 설명 분산: {sum(pca.explained_variance_ratio_)*100:.2f}%")

### 실무 팁: 설명된 분산(Explained Variance) 해석

- **explained_variance_ratio_**: 각 주성분이 원본 데이터의 분산을 얼마나 설명하는지
- PC1이 72.96%를 설명 = 데이터 변동의 73%가 이 축으로 표현됨
- PC1 + PC2 = 95.81% = 4차원 데이터를 2차원으로 압축해도 95%의 정보 유지!

---

## 1.3 PCA 시각화 (Plotly)

In [ ]:
# PCA 결과를 DataFrame으로
df_pca = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'Species': [target_names[i] for i in y]
})

# 2D 산점도
fig = px.scatter(
    df_pca, x='PC1', y='PC2', color='Species',
    title=f'Iris PCA 시각화 (설명 분산: {sum(pca.explained_variance_ratio_)*100:.1f}%)',
    labels={'PC1': f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)',
            'PC2': f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)'},
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(marker=dict(size=10, opacity=0.7))
fig.show()

In [ ]:
# 3D PCA 시각화
pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)

df_pca_3d = pd.DataFrame({
    'PC1': X_pca_3d[:, 0],
    'PC2': X_pca_3d[:, 1],
    'PC3': X_pca_3d[:, 2],
    'Species': [target_names[i] for i in y]
})

fig = px.scatter_3d(
    df_pca_3d, x='PC1', y='PC2', z='PC3', color='Species',
    title=f'Iris 3D PCA (설명 분산: {sum(pca_3d.explained_variance_ratio_)*100:.1f}%)',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(marker=dict(size=6, opacity=0.8))
fig.show()

---

## 1.4 주성분 개수 선택 (Scree Plot)

### Scree Plot (스크리 플롯)

- 각 주성분의 설명 분산을 시각화
- **엘보우 포인트**: 기울기가 급격히 감소하는 지점
- **누적 90% 기준**: 누적 설명 분산이 90%에 도달하는 지점

In [ ]:
# 모든 주성분에 대해 PCA 수행
pca_full = PCA()
pca_full.fit(X_scaled)

# Scree Plot 데이터
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# 시각화
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 개별 분산 (막대)
fig.add_trace(
    go.Bar(
        x=[f'PC{i+1}' for i in range(len(explained_variance))],
        y=explained_variance * 100,
        name='개별 설명 분산',
        marker_color='steelblue'
    ),
    secondary_y=False
)

# 누적 분산 (선)
fig.add_trace(
    go.Scatter(
        x=[f'PC{i+1}' for i in range(len(cumulative_variance))],
        y=cumulative_variance * 100,
        name='누적 설명 분산',
        mode='lines+markers',
        marker_color='coral',
        line=dict(width=3)
    ),
    secondary_y=True
)

# 90% 기준선
fig.add_hline(y=90, line_dash="dash", line_color="green", 
              annotation_text="90% 기준", secondary_y=True)

fig.update_layout(
    title='Scree Plot: 주성분별 설명 분산',
    xaxis_title='주성분',
    yaxis_title='개별 분산 (%)',
    yaxis2_title='누적 분산 (%)',
    legend=dict(x=0.7, y=0.5)
)
fig.show()

### 실무 팁: PCA 주성분 개수 선택 기준

1. **누적 분산 90~95%**: 대부분의 정보 보존
2. **엘보우 규칙**: 기울기 급감 지점
3. **Kaiser 규칙**: 고유값 > 1인 성분만 선택 (표준화 데이터)
4. **교차 검증**: 다운스트림 태스크 성능 기준

---

## 1.5 PCA 성분 해석 (Loading)

In [ ]:
# PCA 성분(로딩) 분석
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=feature_names
)
print("PCA 로딩 (각 원본 특성이 주성분에 기여하는 정도):")
print(loadings.round(3))

In [ ]:
# 로딩 시각화 (화살표)
fig = px.scatter(
    df_pca, x='PC1', y='PC2', color='Species',
    title='PCA Biplot: 주성분과 원본 특성의 관계',
    color_discrete_sequence=px.colors.qualitative.Set2,
    opacity=0.5
)

# 로딩 화살표 추가
scale = 3  # 화살표 크기 조정
for feature, (pc1, pc2) in loadings.iterrows():
    fig.add_annotation(
        x=pc1 * scale, y=pc2 * scale,
        ax=0, ay=0,
        xref="x", yref="y",
        axref="x", ayref="y",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,
        arrowwidth=2,
        arrowcolor="red"
    )
    fig.add_annotation(
        x=pc1 * scale * 1.15,
        y=pc2 * scale * 1.15,
        text=feature.replace(' (cm)', ''),
        showarrow=False,
        font=dict(size=10, color="red")
    )

fig.update_layout(xaxis=dict(range=[-4, 4]), yaxis=dict(range=[-4, 4]))
fig.show()

### 실무 예시: MNIST 손글씨 차원 축소

In [ ]:
# MNIST 손글씨 데이터 로드 (작은 버전)
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"MNIST 데이터: {X_digits.shape[0]}개 샘플, {X_digits.shape[1]}개 특성 (8x8 픽셀)")

In [ ]:
# 표준화 및 PCA
scaler_digits = StandardScaler()
X_digits_scaled = scaler_digits.fit_transform(X_digits)

pca_digits = PCA(n_components=50)  # 64 -> 50
X_digits_pca = pca_digits.fit_transform(X_digits_scaled)

# 누적 분산
cumvar = np.cumsum(pca_digits.explained_variance_ratio_)
n_90 = np.argmax(cumvar >= 0.90) + 1
n_95 = np.argmax(cumvar >= 0.95) + 1

print(f"90% 분산 설명: {n_90}개 성분 필요 (64 -> {n_90}, {(1-n_90/64)*100:.1f}% 압축)")
print(f"95% 분산 설명: {n_95}개 성분 필요 (64 -> {n_95}, {(1-n_95/64)*100:.1f}% 압축)")

In [ ]:
# MNIST 2D 시각화
pca_2d = PCA(n_components=2)
X_digits_2d = pca_2d.fit_transform(X_digits_scaled)

df_digits = pd.DataFrame({
    'PC1': X_digits_2d[:, 0],
    'PC2': X_digits_2d[:, 1],
    'Digit': y_digits.astype(str)
})

fig = px.scatter(
    df_digits, x='PC1', y='PC2', color='Digit',
    title=f'MNIST 손글씨 PCA (설명 분산: {sum(pca_2d.explained_variance_ratio_)*100:.1f}%)',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_traces(marker=dict(size=5, opacity=0.6))
fig.show()

---

# Part 2: t-SNE와 특성 선택

---

## 2.1 t-SNE (t-distributed Stochastic Neighbor Embedding)

### t-SNE란?

- **비선형 차원 축소 기법**: 고차원의 복잡한 구조를 저차원에서 보존
- **주요 용도**: 시각화 전용 (2D/3D)
- **PCA vs t-SNE**:
  - PCA: 선형 변환, 빠름, 전역 구조 보존
  - t-SNE: 비선형, 느림, 지역 구조 보존

### perplexity 파라미터

- **perplexity**: 각 점이 고려하는 이웃의 수 (보통 5~50)
- 작은 값: 지역 구조에 집중, 작은 군집 강조
- 큰 값: 전역 구조에 집중, 큰 군집 강조

In [ ]:
# t-SNE 적용 (Iris)
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_tsne = tsne.fit_transform(X_scaled)

df_tsne = pd.DataFrame({
    'TSNE1': X_tsne[:, 0],
    'TSNE2': X_tsne[:, 1],
    'Species': [target_names[i] for i in y]
})

fig = px.scatter(
    df_tsne, x='TSNE1', y='TSNE2', color='Species',
    title='Iris t-SNE 시각화 (perplexity=30)',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(marker=dict(size=10, opacity=0.7))
fig.show()

In [ ]:
# perplexity 비교
perplexities = [5, 15, 30, 50]

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f'perplexity={p}' for p in perplexities]
)

colors = {'setosa': 'blue', 'versicolor': 'green', 'virginica': 'red'}

for idx, perp in enumerate(perplexities):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    tsne_temp = TSNE(n_components=2, perplexity=perp, random_state=42, n_iter=500)
    X_temp = tsne_temp.fit_transform(X_scaled)
    
    for species_name in target_names:
        mask = [target_names[i] == species_name for i in y]
        fig.add_trace(
            go.Scatter(
                x=X_temp[mask, 0],
                y=X_temp[mask, 1],
                mode='markers',
                name=species_name,
                marker=dict(color=colors[species_name], size=6, opacity=0.7),
                showlegend=(idx == 0)
            ),
            row=row, col=col
        )

fig.update_layout(
    title='t-SNE perplexity 비교',
    height=600,
    legend=dict(x=1.02, y=0.5)
)
fig.show()

### 실무 예시: MNIST t-SNE 시각화

In [ ]:
# MNIST t-SNE (시간이 오래 걸리므로 샘플링)
np.random.seed(42)
sample_idx = np.random.choice(len(X_digits), 500, replace=False)
X_sample = X_digits_scaled[sample_idx]
y_sample = y_digits[sample_idx]

print("t-SNE 변환 중... (약 30초 소요)")
tsne_digits = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_digits_tsne = tsne_digits.fit_transform(X_sample)
print("완료!")

In [ ]:
# MNIST t-SNE 시각화
df_digits_tsne = pd.DataFrame({
    'TSNE1': X_digits_tsne[:, 0],
    'TSNE2': X_digits_tsne[:, 1],
    'Digit': y_sample.astype(str)
})

fig = px.scatter(
    df_digits_tsne, x='TSNE1', y='TSNE2', color='Digit',
    title='MNIST t-SNE (500 samples)',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.show()

### PCA vs t-SNE 비교

In [ ]:
# PCA vs t-SNE 비교 시각화
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['PCA', 't-SNE']
)

# PCA
pca_sample = PCA(n_components=2)
X_sample_pca = pca_sample.fit_transform(X_sample)

for digit in range(10):
    mask = y_sample == digit
    fig.add_trace(
        go.Scatter(
            x=X_sample_pca[mask, 0],
            y=X_sample_pca[mask, 1],
            mode='markers',
            name=str(digit),
            marker=dict(size=5, opacity=0.6),
            legendgroup=str(digit)
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=X_digits_tsne[mask, 0],
            y=X_digits_tsne[mask, 1],
            mode='markers',
            name=str(digit),
            marker=dict(size=5, opacity=0.6),
            legendgroup=str(digit),
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_layout(
    title='MNIST: PCA vs t-SNE 비교',
    height=500,
    legend=dict(x=1.02, y=0.5)
)
fig.show()

### 실무 팁: PCA와 t-SNE 선택 기준

| 기준 | PCA | t-SNE |
|------|-----|-------|
| 속도 | 빠름 | 느림 |
| 용도 | 전처리, 특성 추출 | 시각화 전용 |
| 해석성 | 높음 (로딩 해석 가능) | 낮음 |
| 재현성 | 동일 결과 | 랜덤 시드 필요 |
| 새 데이터 | transform() 가능 | 불가 (재학습 필요) |
| 구조 보존 | 전역 구조 | 지역 구조 |

---

## 2.2 특성 선택 (Feature Selection)

### 특성 선택 vs 차원 축소

| 방법 | 특성 선택 | 차원 축소 (PCA) |
|------|----------|----------------|
| 원리 | 원본 특성 중 선택 | 새로운 특성 생성 |
| 해석성 | 높음 | 낮음 |
| 예시 | [x1, x3, x5] 선택 | PC1 = 0.5*x1 + 0.3*x2 + ... |

### 특성 선택 방법

1. **Filter 방법**: 통계적 테스트 (SelectKBest)
2. **Wrapper 방법**: 모델 기반 반복 선택 (RFE)
3. **Embedded 방법**: 모델 학습 중 선택 (Feature Importance)

In [ ]:
# Wine 데이터셋 사용 (더 많은 특성)
wine = load_wine()
X_wine = wine.data
y_wine = wine.target
wine_features = wine.feature_names

print(f"Wine 데이터: {X_wine.shape[0]}개 샘플, {X_wine.shape[1]}개 특성")
print(f"특성명: {wine_features}")

### 2.2.1 SelectKBest (Filter 방법)

In [ ]:
# SelectKBest: 상위 K개 특성 선택 (ANOVA F-test 기반)
selector = SelectKBest(score_func=f_classif, k=5)
X_kbest = selector.fit_transform(X_wine, y_wine)

# 선택된 특성
selected_features = np.array(wine_features)[selector.get_support()]
scores = selector.scores_

print("SelectKBest 결과 (상위 5개):")
print(f"  선택된 특성: {list(selected_features)}")

In [ ]:
# 모든 특성의 점수 시각화
df_scores = pd.DataFrame({
    'Feature': wine_features,
    'Score': scores
}).sort_values('Score', ascending=True)

fig = px.bar(
    df_scores, x='Score', y='Feature', orientation='h',
    title='SelectKBest: F-Score (ANOVA)',
    color='Score',
    color_continuous_scale='Blues'
)
fig.update_layout(yaxis=dict(categoryorder='total ascending'))
fig.show()

### 2.2.2 RFE (Recursive Feature Elimination)

In [ ]:
# RFE: 재귀적 특성 제거
# 기본 모델로 Logistic Regression 사용
model = LogisticRegression(max_iter=1000, random_state=42)
rfe = RFE(estimator=model, n_features_to_select=5, step=1)
X_rfe = rfe.fit_transform(X_wine, y_wine)

# 선택된 특성
rfe_features = np.array(wine_features)[rfe.support_]
rfe_ranking = rfe.ranking_

print("RFE 결과 (상위 5개):")
print(f"  선택된 특성: {list(rfe_features)}")

In [ ]:
# RFE 순위 시각화
df_rfe = pd.DataFrame({
    'Feature': wine_features,
    'Ranking': rfe_ranking
}).sort_values('Ranking')

fig = px.bar(
    df_rfe, x='Ranking', y='Feature', orientation='h',
    title='RFE: 특성 순위 (1=선택됨)',
    color='Ranking',
    color_continuous_scale='Reds_r'
)
fig.update_layout(yaxis=dict(categoryorder='total descending'))
fig.show()

### 2.2.3 Feature Importance (Random Forest)

In [ ]:
# Random Forest Feature Importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_wine, y_wine)

importances = rf.feature_importances_

df_importance = pd.DataFrame({
    'Feature': wine_features,
    'Importance': importances
}).sort_values('Importance', ascending=True)

fig = px.bar(
    df_importance, x='Importance', y='Feature', orientation='h',
    title='Random Forest: Feature Importance',
    color='Importance',
    color_continuous_scale='Greens'
)
fig.update_layout(yaxis=dict(categoryorder='total ascending'))
fig.show()

### 2.3 특성 선택 방법 비교

In [ ]:
# 세 가지 방법 비교 (상위 5개)
# SelectKBest 상위 5개
kbest_top5 = df_scores.nlargest(5, 'Score')['Feature'].tolist()

# RFE 상위 5개 (ranking=1)
rfe_top5 = list(rfe_features)

# RF Importance 상위 5개
rf_top5 = df_importance.nlargest(5, 'Importance')['Feature'].tolist()

print("특성 선택 방법 비교 (상위 5개):")
print(f"  SelectKBest: {kbest_top5}")
print(f"  RFE:         {rfe_top5}")
print(f"  RF Importance: {rf_top5}")

# 공통 특성 찾기
common = set(kbest_top5) & set(rfe_top5) & set(rf_top5)
print(f"\n모든 방법에서 공통: {list(common)}")

In [ ]:
# 각 특성 선택 방법의 모델 성능 비교
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=1000, random_state=42)

# 전체 특성
score_all = cross_val_score(model, X_wine, y_wine, cv=5).mean()

# SelectKBest 5개
score_kbest = cross_val_score(model, X_kbest, y_wine, cv=5).mean()

# RFE 5개
score_rfe = cross_val_score(model, X_rfe, y_wine, cv=5).mean()

# RF Importance 상위 5개
rf_mask = [f in rf_top5 for f in wine_features]
score_rf = cross_val_score(model, X_wine[:, rf_mask], y_wine, cv=5).mean()

print("로지스틱 회귀 교차검증 정확도:")
print(f"  전체 특성 (13개): {score_all:.4f}")
print(f"  SelectKBest (5개): {score_kbest:.4f}")
print(f"  RFE (5개): {score_rfe:.4f}")
print(f"  RF Importance (5개): {score_rf:.4f}")

In [ ]:
# 성능 비교 시각화
methods = ['전체 (13개)', 'SelectKBest (5개)', 'RFE (5개)', 'RF Importance (5개)']
scores = [score_all, score_kbest, score_rfe, score_rf]

fig = px.bar(
    x=methods, y=scores,
    title='특성 선택 방법별 분류 정확도 비교',
    labels={'x': '방법', 'y': '정확도'},
    color=scores,
    color_continuous_scale='Viridis'
)
fig.update_layout(yaxis=dict(range=[0.9, 1.0]))
fig.show()

### 실무 팁: 특성 선택 전략

1. **빠른 필터링**: SelectKBest로 초기 탐색
2. **최종 선택**: RFE나 모델 기반 중요도로 정교화
3. **앙상블 접근**: 여러 방법의 공통 특성 선택
4. **도메인 지식**: 통계적 결과 + 비즈니스 맥락 고려
5. **교차 검증**: 특성 수에 따른 성능 변화 확인

---

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. 데이터 표준화하기 (난이도: 1/5)

**문제**: Iris 데이터를 StandardScaler로 표준화하고, 표준화 후 평균과 표준편차를 출력하세요.

**힌트**: `StandardScaler().fit_transform()`

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris.data

# 여기에 코드를 작성하세요


### Q2. PCA 2차원 변환 (난이도: 2/5)

**문제**: 표준화된 Iris 데이터를 PCA로 2차원으로 축소하고, 설명된 분산 비율을 출력하세요.

**기대 결과**: 각 주성분의 설명 분산과 누적 설명 분산

In [ ]:
from sklearn.decomposition import PCA

# Q1에서 표준화한 데이터 사용 (X_scaled)
# 여기에 코드를 작성하세요


### Q3. PCA 시각화 (난이도: 2/5)

**문제**: PCA 변환된 Iris 데이터를 px.scatter()로 시각화하세요. 색상은 품종(Species)으로 구분합니다.

**힌트**: DataFrame으로 변환 후 px.scatter() 사용

In [ ]:
import plotly.express as px
import pandas as pd

# Q2의 PCA 결과 사용
# 여기에 코드를 작성하세요


### Q4. Scree Plot 그리기 (난이도: 3/5)

**문제**: Wine 데이터에 대해 모든 주성분의 설명 분산을 막대그래프로, 누적 분산을 선그래프로 그리세요.

**힌트**: `PCA()` (n_components 지정 안 함), `np.cumsum()`, `make_subplots()`

In [ ]:
from sklearn.datasets import load_wine
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

wine = load_wine()
X_wine = wine.data

# 여기에 코드를 작성하세요


### Q5. t-SNE 시각화 (난이도: 3/5)

**문제**: Iris 데이터를 t-SNE로 2차원 변환 후 시각화하세요. perplexity=30 사용.

**힌트**: `TSNE(n_components=2, perplexity=30, random_state=42)`

In [ ]:
from sklearn.manifold import TSNE

# 표준화된 Iris 데이터 사용
# 여기에 코드를 작성하세요


### Q6. perplexity 비교 (난이도: 3/5)

**문제**: perplexity를 [5, 15, 30, 50]으로 변경하며 t-SNE 결과를 2x2 서브플롯으로 비교하세요.

**힌트**: `make_subplots(rows=2, cols=2)`

In [ ]:
perplexities = [5, 15, 30, 50]

# 여기에 코드를 작성하세요


### Q7. SelectKBest 적용 (난이도: 4/5)

**문제**: Wine 데이터에서 SelectKBest로 상위 5개 특성을 선택하고, 각 특성의 F-score를 막대그래프로 시각화하세요.

**힌트**: `SelectKBest(score_func=f_classif, k=5)`, `selector.scores_`

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

wine = load_wine()
X_wine = wine.data
y_wine = wine.target
wine_features = wine.feature_names

# 여기에 코드를 작성하세요


### Q8. RFE 적용 (난이도: 4/5)

**문제**: Logistic Regression을 기반으로 RFE를 적용하여 상위 5개 특성을 선택하고, 특성별 순위를 막대그래프로 시각화하세요.

**힌트**: `RFE(estimator=model, n_features_to_select=5)`, `rfe.ranking_`

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# 여기에 코드를 작성하세요


### Q9. Feature Importance 비교 (난이도: 5/5)

**문제**: Random Forest의 feature_importances_를 추출하고, SelectKBest, RFE와 함께 세 가지 방법의 상위 5개 특성을 비교하세요. 공통 특성을 출력하세요.

**힌트**: 집합(set)의 교집합 `&` 연산

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 여기에 코드를 작성하세요


### Q10. 종합 문제: 차원 축소 파이프라인 (난이도: 5/5)

**문제**: MNIST 데이터(digits)에 대해:
1. StandardScaler로 표준화
2. PCA로 90% 분산을 설명하는 주성분 수 찾기
3. 해당 수로 차원 축소 후 Logistic Regression 정확도 측정
4. 원본(64차원) vs 축소된 데이터의 정확도 비교

**힌트**: `np.cumsum(pca.explained_variance_ratio_) >= 0.9`, `cross_val_score()`

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score

digits = load_digits()
X_digits = digits.data
y_digits = digits.target

# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: PCA 핵심 요약

| 개념 | 핵심 내용 | 코드 |
|------|----------|------|
| 차원의 저주 | 고차원에서 거리 의미 상실, 과적합 위험 | - |
| PCA 원리 | 분산 최대화 축 찾기, 직교 변환 | `PCA(n_components=k)` |
| 설명 분산 | 각 PC가 원본 정보를 얼마나 보존 | `pca.explained_variance_ratio_` |
| 주성분 선택 | 누적 90%+ 또는 엘보우 규칙 | `np.cumsum()` |
| PCA 로딩 | 원본 특성과 PC의 관계 | `pca.components_` |

### Part 2: t-SNE와 특성 선택 핵심 요약

| 방법 | 특징 | 코드 |
|------|------|------|
| t-SNE | 비선형, 시각화 전용, 지역 구조 보존 | `TSNE(perplexity=30)` |
| SelectKBest | 통계적 필터링 (F-test) | `SelectKBest(f_classif, k=5)` |
| RFE | 모델 기반 재귀적 제거 | `RFE(estimator, n_features_to_select)` |
| Feature Importance | 트리 기반 중요도 | `rf.feature_importances_` |

### 실무 팁

1. **PCA 전 반드시 표준화**: 스케일 차이가 크면 결과 왜곡
2. **t-SNE는 시각화 전용**: 새 데이터 변환 불가, 재현성 위해 random_state 설정
3. **특성 선택 앙상블**: 여러 방법의 공통 특성이 가장 신뢰도 높음
4. **perplexity 튜닝**: 데이터 크기에 따라 조정 (작은 데이터는 작은 값)
5. **교차 검증 필수**: 특성 수에 따른 성능 변화 확인